In [ ]:
import pandas as pd
import sqlite3
from datetime import datetime
import hashlib

def make_remark_id(model_name, strategy):
    unique_string = f"{model_name}_{strategy}_{datetime.now()}"
    return hashlib.md5(unique_string.encode()).hexdigest()
    
conn = sqlite3.connect("../data/automated_metrics.db")
conn.execute("PRAGMA foreign_keys = ON;")
cursor = conn.cursor()

cases_df = pd.read_sql_query("SELECT * from hundred_sampled_transcript_and_context;", conn)
remark_addition_query = "INSERT INTO remark (remark_id, model, prompting_strategy, justice, " \
                        "remark_text, log_id, context_id) VALUES (?, ?, ?, ?, ?, ?, ?)"
additional_remarks = []
for index, row in cases_df.iterrows():
    generated_remark = row['turn_text']
    model_name = 'actual_text'
    log_id = 'manual'
    strategy = None
    remark_id = make_remark_id(model_name, strategy)
    additional_remarks.append((remark_id, model_name, strategy, row["justice"], generated_remark, 
                               log_id, row["context_id"]))

cursor.executemany(remark_addition_query, additional_remarks)
conn.commit()
cursor.close()
conn.close()

For the adversarial metrics table

In [2]:
import pandas as pd
import sqlite3
from datetime import datetime
import hashlib

def make_remark_id(model_name, strategy):
    unique_string = f"{model_name}_{strategy}_{datetime.now()}"
    return hashlib.md5(unique_string.encode()).hexdigest()
    
conn = sqlite3.connect("../data/adversarial_metrics.db")
conn.execute("PRAGMA foreign_keys = ON;")
cursor = conn.cursor()

cases_df = pd.read_sql_query("SELECT * from transcript_and_context;", conn)
remark_addition_query = "INSERT INTO remark (remark_id, model, prompting_strategy, justice, " \
                        "remark_text, log_id, context_id) VALUES (?, ?, ?, ?, ?, ?, ?)"
additional_remarks = []
for index, row in cases_df.iterrows():
    generated_remark = row['turn_text']
    model_name = 'actual_text'
    log_id = 'manual'
    strategy = None
    remark_id = make_remark_id(model_name, strategy)
    additional_remarks.append((remark_id, model_name, strategy, row["justice"], generated_remark, 
                               log_id, row["context_id"]))

cursor.executemany(remark_addition_query, additional_remarks)
conn.commit()
cursor.close()
conn.close()

Create adversarial metrics view

In [6]:
write_conn = sqlite3.connect("../data/adversarial_metrics.db")
write_conn.execute("PRAGMA foreign_keys = ON;")
cur = write_conn.cursor()

def create_remark_transcript_context_view(new_view_name, transcript_table, context_table, remark_table):
    create_view_sql = f"""CREATE VIEW IF NOT EXISTS {new_view_name} AS
    SELECT
        {remark_table}.remark_id as remark_id,
        {remark_table}.justice as justice,
        {remark_table}.remark_text as remark_text,
        {remark_table}.log_id as remark_log_id,
        {context_table}.context as context,
        {transcript_table}.case_facts as case_facts,
        {transcript_table}.legal_question as legal_question
    FROM remark
    JOIN context ON remark.context_id = context.context_id
    JOIN transcript ON context.transcript_id = transcript.transcript_id;
    """
    cur.execute(create_view_sql)
    write_conn.commit()

create_remark_transcript_context_view("remark_transcript_context", "transcript", "context", "remark")